# **<u>Proyecto de automatización de trabajo: iteración sobre Sheets en carpetas de Google Drive, y append en otra Sheets de BBDD consolidada.</u>**

---

## <span style="background-color: #FFC0CB">Pending:</span>
            1. establecer un trigger semanal
          

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importación de bibliotecas y montaje de drive

In [25]:
import os
import time
import pandas as pd
from google.colab import drive
import gspread
from gspread_dataframe import set_with_dataframe  # Agrega esta línea
from oauth2client.service_account import ServiceAccountCredentials
from oauth2client.client import GoogleCredentials
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Inicialización de credenciales de Google, Google API y archivo JSON

### **Atención**: previamente se debe crear un proyecto en la nube y seguir una serie de pasos para autenticar el acceso a sus archivos de drive. Más información [aquí](https://cloud.google.com/gcp/?hl=es&utm_source=bing&utm_medium=cpc&utm_campaign=emea-es-all-es-bkws-all-all-trial-e-gcp-1011340&utm_content=text-ad-none-any-DEV_c-CRE_-ADGP_Hybrid+%7C+BKWS+-+EXA+%7C+Txt+-+GCP+-+General+-+v3-KWID_43700061311461132-kwd-77240890777560:loc-170-userloc_164782&utm_term=KW_google%20cloud-NET_o-PLAC_&&refclickid=NNE_HotelDigitalMediaCampaign&pmed=DPM_CVG_TRV_HOT_NNE_SEM_GOG_&gclid=e13bffff37a5160b7e7e99c1ad5ecd04&gclsrc=3p.ds&+)


In [27]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Definir ámbito de actuación y credenciales
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP (automat)/Automatización_23-09-2024/test-automation-gsheets-5fa673850271.json', scope)

# Autenticarse con Google Sheets API
client = gspread.authorize(creds)

### Pre-procesamiento de datos:

* Nulos
* Columnas a descartar

In [ ]:
# df.isna().sum(axis=1)

In [ ]:
# pip install --upgrade google-auth-oauthlib google-auth-httplib2 pandas gspread

In [31]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Configuración de las credenciales
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP (automat)/Automatización_23-09-2024/test-automation-gsheets-5fa673850271.json', scope)
client = gspread.authorize(creds)

# Acceder a la hoja de cálculo
sheet = client.open_by_key('1ajE1kRuJaEY5T1IhI1fcerRiaPtsC94LZSvYXkZ1VwQ')

# Obtener la primera hoja
worksheet = sheet.get_worksheet(0)
print(worksheet.get_all_records())


Output hidden; open in https://colab.research.google.com to view.

##Proceso de iteración por la carpeta de boletines parlamentarios, para identificar y levantar solamente los archivos que hayan sido modificados o subidos después de cierta fecha. Luego, los inserta debajo de la hoja correspondiente de base de datos unificada

In [40]:
import os
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

# Configuración
sheet_name = 'Boletines_unificado'
sheet_id_unificado = '1ajE1kRuJaEY5T1IhI1fcerRiaPtsC94LZSvYXkZ1VwQ'
CWD = '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP (automat)/Boletines Parlamentarios'
cutoff_date = datetime(2024, 9, 23)  # Fecha de corte

# Credenciales de acceso a la API de Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP (automat)/Automatización_23-09-2024/test-automation-gsheets-5fa673850271.json', scope)
client = gspread.authorize(creds)

# def limpiar_dataframe(dfp):
#     dfp['Fecha publicación'] = dfp['Fecha publicación'].astype(str)
#     dfp_cleaned = dfp.fillna('')
#     data_cleaned = dfp_cleaned.values.tolist()
#     return data_cleaned

def limpiar_dataframe(dfp):
    # Convertir la columna 'Fecha publicación' a tipo datetime y formatearla
    dfp['Fecha publicación'] = pd.to_datetime(dfp['Fecha publicación'], errors='coerce').dt.strftime('%d/%m/%Y')

    # Rellenar valores nulos y convertir a lista de listas
    dfp_cleaned = dfp.fillna('')  # Rellenar los NaN con cadenas vacías
    data_cleaned = dfp_cleaned.values.tolist()  # Convertir el DataFrame a lista de listas
    return data_cleaned

def main():
    files = os.listdir(CWD)
    dataframes = []

    for file_ in files:
        file_path = os.path.join(CWD, file_)

        if os.path.isfile(file_path) and file_.endswith('.xlsx'):
            # Obtener la fecha de modificación del archivo
            file_mod_time = datetime.fromtimestamp(os.path.getmtime(file_path))

            # Procesar solo los archivos modificados después de la fecha de corte
            if file_mod_time > cutoff_date:
                # Leer el archivo de Excel
                dfp = pd.read_excel(file_path)
                dataframes.append(dfp)
                print("DataFrame creado para el archivo:", file_)
                print(dfp)

                # Limpiar el DataFrame
                data_cleaned = limpiar_dataframe(dfp)

                # Obtener el nombre del archivo sin la extensión
                filename_no_ext = os.path.splitext(file_)[0]

                # Obtener el cliente de Google Sheets y abrir el archivo unificado
                sheet = client.open_by_key(sheet_id_unificado)

                # Acceder a la segunda hoja del archivo unificado
                worksheet_boletines_parlamentarios = sheet.get_worksheet(1)

                # Obtener el índice de la fila donde deseas insertar los datos
                indice_fila_insercion = len(worksheet_boletines_parlamentarios.get_all_values()) + 1
                print("Índice de fila de inserción:", indice_fila_insercion)

                # Insertar los datos limpios en la segunda hoja del archivo unificado
                worksheet_boletines_parlamentarios.insert_rows(data_cleaned, indice_fila_insercion)
                print("Datos insertados correctamente en la fila", indice_fila_insercion)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


DataFrame creado para el archivo: 240926 Boletines Parlamentarios.xlsx
       ID             Sector         Sector2  Subsector  Tema Legislatura  \
0   11828          Movilidad             NaN        NaN   NaN          XV   
1   11829  Economía_Circular             NaN        NaN   NaN          XV   
2   11830      Medioambiente             NaN        NaN   NaN          XV   
3   11831           Primario             NaN        NaN   NaN          XV   
4   11832            Turismo             NaN        NaN   NaN          XV   
..    ...                ...             ...        ...   ...         ...   
70  11898     Digitalización             NaN        NaN   NaN         XII   
71  11899           Comercio             NaN        NaN   NaN         XII   
72  11900             Sequía             NaN        NaN   NaN         XII   
73  11901           Primario  Digitalización        NaN   NaN         XII   
74  11902  Economía_Circular             NaN        NaN   NaN         XII   

   M

## Mismo proceso de iteración por la carpeta de boletines autonómicos, pero esta vez identifica y levanta solamente los archivos que hayan sido modificados o subidos después de cierta fecha. Luego, los inserta debajo de la hoja correspondiente de base de datos unificada

In [42]:
import os
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

sheet_name = 'Boletines_unificado'
sheet_id_unificado = '1ajE1kRuJaEY5T1IhI1fcerRiaPtsC94LZSvYXkZ1VwQ'
CWD = '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP (automat)/Boletines Autonómicos'
cutoff_date = datetime(2024, 9, 23)  # Fecha de corte

# Credenciales de acceso a la API de Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP (automat)/Automatización_23-09-2024/test-automation-gsheets-5fa673850271.json', scope)
client = gspread.authorize(creds)

def limpiar_dataframe(dfp):
    # Convertir la columna 'Fecha publicación' a tipo datetime y formatearla
    dfp['Fecha de publicación'] = pd.to_datetime(dfp['Fecha de publicación'], errors='coerce').dt.strftime('%d/%m/%Y')

    # Rellenar valores nulos y convertir a lista de listas
    dfp_cleaned = dfp.fillna('')  # Rellenar los NaN con cadenas vacías
    data_cleaned = dfp_cleaned.values.tolist()  # Convertir el DataFrame a lista de listas
    return data_cleaned

def main():
    files = os.listdir(CWD)
    dataframes = []

    for file_ in files:
        file_path = os.path.join(CWD, file_)

        if os.path.isfile(file_path) and file_.endswith('.xlsx'):
            # Obtener la fecha de modificación del archivo
            file_mod_time = datetime.fromtimestamp(os.path.getmtime(file_path))

            # Procesar solo los archivos modificados después de la fecha de corte
            if file_mod_time > cutoff_date:
                # Leer el archivo de Excel con el motor especificado
                dfp = pd.read_excel(file_path, engine='openpyxl')
                dataframes.append(dfp)
                print("DataFrame creado para el archivo:", file_)
                print(dfp)

                # Limpiar el DataFrame
                data_cleaned = limpiar_dataframe(dfp)

                # Obtener el nombre del archivo sin la extensión
                filename_no_ext = os.path.splitext(file_)[0]

                # Obtener el cliente de Google Sheets y abrir el archivo unificado
                client = gspread.authorize(creds)
                sheet = client.open_by_key(sheet_id_unificado)

                # Acceder a la segunda hoja del archivo unificado
                worksheet_boletines_autonomicos = sheet.get_worksheet(0)

                # Obtener el índice de la fila donde deseas insertar los datos
                indice_fila_insercion = len(worksheet_boletines_autonomicos.get_all_values()) + 1
                print("Índice de fila de inserción:", indice_fila_insercion)

                # Insertar los datos limpios en la segunda hoja del archivo unificado
                worksheet_boletines_autonomicos.insert_rows(data_cleaned, indice_fila_insercion)
                print("Datos insertados correctamente en la fila", indice_fila_insercion)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


DataFrame creado para el archivo: 240926 Boletines Autonómicos.xlsx
      ID         Sector       Sector2         Sector3  \
0   2349       Primario        Sequía             NaN   
1   2350   Alimentación      Primario             NaN   
2   2351        Empresa           NaN             NaN   
3   2352        Empresa    Innovación  Digitalización   
4   2353        Empresa  Alimentación        Primario   
5   2354        Empresa           NaN             NaN   
6   2355   Farmacéutico         Salud             NaN   
7   2356  Institucional           NaN             NaN   
8   2357  Institucional           NaN             NaN   
9   2358  Institucional           NaN             NaN   
10  2359  Institucional           NaN             NaN   
11  2360  Institucional           NaN             NaN   
12  2361  Institucional           NaN             NaN   
13  2362  Institucional           NaN             NaN   
14  2363  Institucional           NaN             NaN   
15  2364  Instituci

## Mismo proceso de iteración por las carpeta de agendas parlamentarias, pero esta vez identifica y levanta solamente los archivos que hayan sido modificados o subidos después de cierta fecha. Luego, los inserta debajo de la hoja correspondiente de base de datos unificada

In [46]:
import os
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

sheet_name = 'Boletines_unificado'
sheet_id_unificado = '1ajE1kRuJaEY5T1IhI1fcerRiaPtsC94LZSvYXkZ1VwQ'
CWD = '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP (automat)/Agendas parlamentarias'
cutoff_date = datetime(2024, 9, 23)  # Fecha de corte

# Credenciales de acceso a la API de Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP (automat)/Automatización_23-09-2024/test-automation-gsheets-5fa673850271.json', scope)
client = gspread.authorize(creds)

def limpiar_dataframe(dfp):
    # Convertir la columna 'Fecha publicación' a tipo datetime y formatearla
    dfp['Fecha convocada'] = pd.to_datetime(dfp['Fecha convocada'], errors='coerce').dt.strftime('%d/%m/%Y')

    # Rellenar valores nulos y convertir a lista de listas
    dfp_cleaned = dfp.fillna('')  # Rellenar los NaN con cadenas vacías
    data_cleaned = dfp_cleaned.values.tolist()  # Convertir el DataFrame a lista de listas
    return data_cleaned

def main():
    files = os.listdir(CWD)
    dataframes = []
    for file_ in files:
        file_path = os.path.join(CWD, file_)
        if os.path.isfile(file_path) and file_.endswith('.xlsx'):
          file_mod_time = datetime.fromtimestamp(os.path.getmtime(file_path))

# Procesar solamente los archivos modificados depsués de la fecha de corte
          if file_mod_time > cutoff_date:
            dfp = pd.read_excel(file_path)
            dataframes.append(dfp)
            print("DataFrame creado para el archivo:", file_)
            print(dfp)

            # Limpiar el DataFrame
            data_cleaned = limpiar_dataframe(dfp)

            # Obtener el nombre del archivo sin la extensión
            filename_no_ext = os.path.splitext(file_)[0]

            client = gspread.authorize(creds)
            sheet = client.open_by_key(sheet_id_unificado)

            # Acceder a la segunda hoja del archivo unificado
            worksheet_agendas_parlamentarias = sheet.get_worksheet(2)

            # Obtener el índice de la fila donde deseas insertar los datos
            indice_fila_insercion = len(worksheet_agendas_parlamentarias.get_all_values()) + 1
            print("Índice de fila de inserción:", indice_fila_insercion)

            # Insertar los datos limpios en la segunda hoja del archivo unificado
            worksheet_agendas_parlamentarias.insert_rows(data_cleaned, indice_fila_insercion)
            print("Datos insertados correctamente en la fila", indice_fila_insercion)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


DataFrame creado para el archivo: 240923_Agendas Parlamentarias.xlsx
      ID         Sector     Sector2 Subsector              Tema Legislatura  \
0   2732        Turismo         NaN       NaN               NaN          XI   
1   2733        Turismo    Vivienda       NaN               NaN          XI   
2   2734        Turismo         NaN       NaN               NaN          XI   
3   2735        Turismo         NaN       NaN    Sostenibilidad          XI   
4   2736          Salud         NaN       NaN  Listas de Espera          XI   
..   ...            ...         ...       ...               ...         ...   
62  2794        Energía         NaN       NaN               NaN          XI   
63  2795  Institucional   Movilidad       NaN               NaN        XIII   
64  2796  Institucional  Innovación       NaN               NaN        XIII   
65  2797  Institucional     Turismo       NaN               NaN        XIII   
66  2798  Institucional    Primario       NaN               Na

# **Et... voilà!**